# 全连接神经网络进行鸢尾花分类

问题0:类别标签为onehot编码的分类问题,如何选择损失函数?  answer:categorical_crossentropy

问题1:分类问题中如何让输出层神经元输出整数?  answer:没有必要让它输出整数,只要输出分数,按照分数高低分类

## 准备数据

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import *
from sklearn.preprocessing import LabelBinarizer
from sklearn.cross_validation import train_test_split
import tensorflow as tf

D:\study\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
D:\study\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dataSet = make_classification(n_classes=3, n_features=4, n_clusters_per_class=1, n_samples=1500, random_state=2018, n_redundant=1, shuffle=1)
dataSet

(array([[-0.9786559 , -1.93511269,  1.67048429, -1.35191153],
        [-0.68309351, -1.21060004,  1.26434357, -0.27003405],
        [-1.23531632, -2.31352805,  2.19921277, -0.98677152],
        ...,
        [ 0.1837119 , -1.60095609, -1.6926683 , -0.82385412],
        [ 0.02756183,  1.12525576,  0.70474037,  0.03069563],
        [ 0.30295649, -1.85502026, -2.24013493,  0.50985887]]),
 array([0, 0, 0, ..., 1, 2, 1]))

In [3]:
raw_y = LabelBinarizer().fit_transform(dataSet[1])
raw_y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0]])

In [4]:
train_x, test_x, train_y, test_y = train_test_split(dataSet[0], raw_y, test_size=0.1)

## 搭建模型

In [5]:
def add_layer(in_data, num_input, num_output, active_function=None):   #num_input:特征数, num_output:神经元数
    weights = tf.Variable(tf.random_normal((num_input, num_output)))
    bias = tf.Variable(tf.zeros((1, num_output)) + 0.5)
    result = tf.matmul(in_data, weights) + bias
    
    if active_function == None:
        return result
    else:
        return active_function(result)    #节点激活函数

In [6]:
def train_model(num_neural=6):
    num_feat = len(train_x[0])
    num_class = len(set(dataSet[1]))
    
    input_x = tf.placeholder(tf.float32, [None, num_feat])
    input_y = tf.placeholder(tf.float32, [None, num_class])
    
    lay_0 = add_layer(input_x, num_feat, num_neural)
    lay_pred = add_layer(lay_0, num_neural, num_class, tf.nn.relu)

    loss = tf.reduce_mean(tf.reduce_sum(abs(input_y - lay_pred), reduction_indices=1))
    train_step = tf.train.GradientDescentOptimizer(0.08).minimize(loss)

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for step in range(3001):
            sess.run(train_step, feed_dict={input_x: train_x, input_y: train_y})
            if step % 50 == 0:
                print(sess.run(loss, feed_dict={input_x: train_x, input_y: train_y}))
                
        print(sess.run(lay_pred, feed_dict={input_x: test_x}))
                
train_model()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
7.937688
0.6709931
0.69711745
0.78699887
0.7461545
0.7217724
0.6158357
0.694033
0.62451494
0.6431552
0.67680174
0.5761509
0.63833725
0.60504967
0.60345954
0.5939377
0.5805626
0.5745242
0.57261133
0.5769394
0.56574744
0.56933904
0.5663217
0.56627154
0.5608067
0.5625608
0.5613058
0.5587144
0.5595982
0.55707026
0.5614451
0.555825
0.5584746
0.5531561
0.5572551
0.5537254
0.5543703
0.5562194
0.553576
0.55433667
0.5555692
0.5541137
0.5539286
0.55383307
0.55273736
0.5537486
0.55238324
0.55235636
0.55332756
0.5518653
0.5527163
0.55308354
0.55359226
0.55128163
0.55169773
0.55325174
0.5520792
0.55154485
0.5514998
0.5536942
0.5517281
[[0.         0.9840207  0.        ]
 [0.         0.2161853  0.        ]
 [0.68059045 0.         0.        ]
 [0.         0.29299822 0.        ]
 [0.         0.         0.8399723 ]
 [0.         0.         0.4304938 ]
 [0.         0.         0.94145435]
 [0.47530484 0.         0.        ]
 [0.    

输出层神经元应该输出整数的